Anime Recomendation Datasbase 2020


***Descripión***:

En esta notebook intentamos obtener los hiperparámetros que mejor ajusten nuestro modelo LightFM.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pickle

In [ ]:
path_pickle="/content/drive/MyDrive/TP FINAL ICARO/matriz_interacciones.pickle"

In [ ]:
path_test="/content/drive/MyDrive/TP FINAL ICARO/interacciones_test.pickle"

In [ ]:
with open(path_pickle, 'rb') as f:
    interacciones_train = pickle.load(f)

In [ ]:
with open(path_test, 'rb') as f:
    interacciones_test = pickle.load(f)

In [ ]:
pip install scikit-learn lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808332 sha256=c3f5758b2a88f71ba86fabccc2141b84166da3a894e776c166f8d390225a9fc1
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:
from lightfm import LightFM

In [ ]:
from lightfm.evaluation import precision_at_k

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [ ]:
param_grid = {
    'learning_rate': [0.0009,0.009,0.09],
    'loss': ['logistic', 'warp'],
    'no_components': [1,5]
}

In [ ]:
def custom_scorer(model, train_coo, test_coo, k=5):
    return precision_at_k(model, interacciones_test, interacciones_train, k=k).mean()

In [ ]:
precision_scorer = make_scorer(custom_scorer)

In [ ]:
model = LightFM()

In [ ]:
grid_search = GridSearchCV(model, param_grid, cv=5,scoring=precision_scorer)
grid_search.fit(interacciones_train)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


GridSearchCV(cv=5, estimator=<lightfm.lightfm.LightFM object at 0x7a0792855ae0>,
             param_grid={'learning_rate': [0.0009, 0.009, 0.09],
                         'loss': ['logistic', 'warp'],
                         'no_components': [1, 5]},
             scoring=make_scorer(custom_scorer))

In [ ]:
print("Mejores parámetros encontrados:", grid_search.best_params_)

Mejores parámetros encontrados: {'learning_rate': 0.0009, 'loss': 'logistic', 'no_components': 1}


In [ ]:
best_model= LightFM(learning_rate=0.0009, loss='logistic', no_components=1)

In [ ]:
best_model.fit(interacciones_train)

In [ ]:
precision = precision_at_k(best_model, interacciones_test, interacciones_train, k=20).mean()

print("Precision at K en los datos de prueba con el mejor modelo:", precision)

Precision at K en los datos de prueba con el mejor modelo: 0.28676036
